<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/224-LangChain-ChatBot-H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install gradio
!pip install langchain langchain_community
!pip install -qU langchain-openai

In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
import gradio as gr
from datetime import datetime

# Initialize the chat model
llm = ChatOpenAI(model="gpt-3.5-turbo")

감정 분석 기능:
유저의 메시지에 대한 감정 분석을 수행하여, 챗봇의 응답을 감정에 따라 조절할 수 있습니다.

In [5]:
from textblob import TextBlob
import gradio as gr
from langchain.schema import HumanMessage, AIMessage

def predict(message, history):
    if history is None:
        history = []

    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    # Adding only the current human message to the format history
    history_langchain_format.append(HumanMessage(content=message))

    # Generate GPT response
    gpt_response = llm(history_langchain_format)

    # Analyzing the sentiment of the human message
    human_sentiment = TextBlob(message).sentiment
    if human_sentiment.polarity < 0:
        gpt_response.content = "It's sad to hear such stories." + gpt_response.content

    # Analyzing the sentiment of the AI response
    ai_sentiment = TextBlob(gpt_response.content).sentiment
    if ai_sentiment.polarity < 0:
        gpt_response.content = gpt_response.content + " I hope this helps."

    # Append the human message and AI response to history
    history.append((message, gpt_response.content))

    return gpt_response.content, history

# Define Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=["text", "state"],
    outputs=["text", "state"],
    live=False  # Turn off streaming
)

# Launch the interface
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6208108341f66bcde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
